In [203]:
import pandas as pd
pd.options.mode.chained_assignment = None

In [204]:
octo = pd.read_csv('bad-scrape.raw.csv', names=[i for i in range(0, 4)])

In [205]:
octo_test = octo.copy()

In [206]:
# blank col, name col, value col, blank col
rows = octo_test.loc[octo_test[0].isnull() & octo_test[3].isnull()].index.values
octo_test.iloc[rows, 0] = octo_test.iloc[rows][1]
octo_test.iloc[rows, 1] = octo_test.iloc[rows][2]

In [207]:
# blank col, name col, blank col, value col
rows = octo_test.loc[octo_test[0].isnull() & octo_test[2].isnull()].index.values
octo_test.iloc[rows, 0] = octo_test.iloc[rows][1]
octo_test.iloc[rows, 1] = octo_test.iloc[rows][3]

In [208]:
# blank col, blank col, name col, value col
rows = octo_test.loc[octo_test[0].isnull() & octo_test[1].isnull()].index.values
octo_test.iloc[rows, 0] = octo_test.iloc[rows][2]
octo_test.iloc[rows, 1] = octo_test.iloc[rows][3]

In [209]:
# name col, blank col, blank col, value col
octo_test.loc[octo_test[1].isnull() & octo_test[2].isnull(), 1] = octo_test.loc[octo_test[1].isnull() & octo_test[2].isnull()][3]

In [210]:
# name col, blank col, value col, blank col
octo_test.loc[octo_test[0].notnull() & octo_test[1].isnull() & octo_test[2].notnull(), 1] = octo_test.loc[octo_test[0].notnull() & octo_test[1].isnull() & octo_test[2].notnull()][2]

In [211]:
# remove indoor/outdoor rows
octo_test[0] = octo_test[0].str.rstrip()
octo_test.drop(list(octo_test.loc[octo_test[0].isin(['Indoor', 'Outdoor', 'Oudoor'])].index.values), inplace=True)

# remove dummy cols
octo_test.drop([2, 3], axis=1, inplace=True)

# reset the index
octo_test.reset_index(inplace=True, drop=True)

In [212]:
def process(df):
    # cleanup
    df.rename(columns={0: 'fixture_location', 1: 'result_value'}, inplace=True)
    df['park_name'] = ['' for i in range(0, len(df))]
    
    # get park names
    indices = df.loc[df.result_value.isnull()].index
    names = df.loc[df.result_value.isnull()].fixture_location
    parks = list(zip(indices, names))

    # restructure
    for ind, park in enumerate(parks):
        start, park = park
        try:
            end = parks[ind+1][0] - 1
        except IndexError:
            end = len(df)
        df.loc[start:end, 'park_name'] = park
        
    df.drop(list(indices), inplace=True)
    df['result_flag'] = df['result_value'].apply(lambda x: x.split(' ')[-1] if len(x.split(' ')) > 1 else '')
    df['result_value'] = df['result_value'].apply(lambda x: x.split(' ')[0]).str.replace('˂', '<')

In [213]:
process(octo_test)
octo_test = octo_test.ix[:, [2, 0, 1, 3]]

In [214]:
# octo_test.to_csv('chi-parks-lead-results-10-2016.processed.csv', index=False)

In [215]:
# octo_test.loc[octo_test['fixture_location'].str.contains('\*'), 'offline'] = 'Y'
# octo_test.loc[octo_test['result_value'].str.contains('<'), 'threshold'] = 'Y'

In [216]:
# clean up two rows w/o space between result value and result flag
octo_test.iloc[1002]['result_value'] = '3.09'
octo_test.iloc[1002]['result_flag'] = 'J'
octo_test.iloc[1002]['processed_result'] = 3.09

octo_test.iloc[2020]['result_value'] = '0.243'
octo_test.iloc[2020]['result_flag'] = 'J'
octo_test.iloc[2020]['processed_result'] = 0.243

In [217]:
octo_test['processed_result'] = octo_test['result_value'].apply(lambda x: 0 if '<' in x else pd.to_numeric(x))

In [218]:
octo_test['exceeds_epa'] = octo_test['processed_result'].apply(lambda x: 1 if x > 15 else 0)

In [219]:
octo_test

,park_name,fixture_location,result_value,result_flag,processed_result,exceeds_epa
1,Abbott,Fitness Center,<0.120,U,0.000,0
2,Abbott,Main Lobby,0.579,J,0.579,0
3,Abbott,Fountain 1 High,<0.120,U,0.000,0
4,Abbott,Fountain 1 Low,<0.120,U,0.000,0
5,Abbott,Fountain 2,<0.120,U,0.000,0
6,Abbott,Fountain 3,0.245,J,0.245,0
7,Abbott,Fountain 4,4.48,J,4.480,0
9,Ada,Main Lobby,<2.0,U,0.000,0
10,Ada,Kitchen Sink,<2.0,U,0.000,0
11,Ada,Fountain 1,11.7,,11.700,0


handle single case of [follow up sample](https://github.com/datamade/chicago-lead/blob/parks-processing/assets/chi-parks-lead-results/chi-parks-lead-results-10-2016.processed.csv#L1118)